In [520]:
from pydub import AudioSegment
import os

def overlay_audios(audio_files, output_file):
    combined = AudioSegment.from_file(audio_files[0])
    for file in audio_files[1:]:
        audio = AudioSegment.from_file(file)
        combined = combined.overlay(audio)
    
    output_dir = os.path.dirname(output_file)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    combined.export(output_file, format="ogg")

In [522]:
import pandas as pd

df = pd.read_csv('./open/train.csv')
df["path"]

0        ./train/RUNQPNJF.ogg
1        ./train/JFAWUOGJ.ogg
2        ./train/RDKEKEVX.ogg
3        ./train/QYHJDOFK.ogg
4        ./train/RSPQNHAO.ogg
                 ...         
55433    ./train/NJGPMAJL.ogg
55434    ./train/SZXIACUZ.ogg
55435    ./train/PXLBTGRH.ogg
55436    ./train/CGGQGPOQ.ogg
55437    ./train/CPMKEDIS.ogg
Name: path, Length: 55438, dtype: object

In [524]:
# fake면 0, real이면 1
train_path_org = []
for i in df["path"]:
    train_path_org.append(os.path.join("./open", i ))
    
label = []
for i in df["label"]:
    if i=="fake":
        label.append(0)
    else:
        label.append(1)
df["path"] = train_path_org
df["label"]=label

In [526]:
df.head()

,id,path,label
0,RUNQPNJF,./open/./train/RUNQPNJF.ogg,1
1,JFAWUOGJ,./open/./train/JFAWUOGJ.ogg,0
2,RDKEKEVX,./open/./train/RDKEKEVX.ogg,1
3,QYHJDOFK,./open/./train/QYHJDOFK.ogg,1
4,RSPQNHAO,./open/./train/RSPQNHAO.ogg,1


In [528]:
import os
import random
import string
from tqdm import tqdm


# 랜덤하게 2가지 화자 섞어서 데이터 생성

new_data = []

for _ in tqdm(range(30000)):
    
    sample = df.sample(2)
    ids = sample['id'].tolist()
    paths = sample['path'].tolist()
    labels = sample['label'].tolist()

    # 무작위 7대문자 생성
    random_id = 'made_' + ''.join(random.choices(string.ascii_uppercase, k=7))
    
    # 저장할 경로 생성
    output_file = f"./open/train_overlay/{random_id}.ogg"
    
    # 오버레이 및 파일 저장
    overlay_audios(paths, output_file)
    
    # 새로운 데이터프레임에 추가할 정보 생성
    new_entry = {
        'id': random_id,
        'path': output_file,
        'label_1': labels[0],
        'label_2': labels[1]
    }
    new_data.append(new_entry)

# 새로운 데이터프레임 생성
new_df = pd.DataFrame(new_data)

# 결과 확인
new_df.head()

100%|███████████████████████████████████| 20000/20000 [1:43:28<00:00,  3.22it/s]


,id,path,label_1,label_2
0,made_ZVPLMJZ,./open/train_overlay/made_ZVPLMJZ.ogg,0,1
1,made_WVOSCQM,./open/train_overlay/made_WVOSCQM.ogg,0,1
2,made_AWFBVJI,./open/train_overlay/made_AWFBVJI.ogg,1,1
3,made_CNOQHMY,./open/train_overlay/made_CNOQHMY.ogg,1,1
4,made_JWTCJWJ,./open/train_overlay/made_JWTCJWJ.ogg,1,1


In [530]:
new_df.head(20)

,id,path,label_1,label_2
0,made_ZVPLMJZ,./open/train_overlay/made_ZVPLMJZ.ogg,0,1
1,made_WVOSCQM,./open/train_overlay/made_WVOSCQM.ogg,0,1
2,made_AWFBVJI,./open/train_overlay/made_AWFBVJI.ogg,1,1
3,made_CNOQHMY,./open/train_overlay/made_CNOQHMY.ogg,1,1
4,made_JWTCJWJ,./open/train_overlay/made_JWTCJWJ.ogg,1,1
5,made_BMGBIUB,./open/train_overlay/made_BMGBIUB.ogg,0,1
6,made_XDMACLJ,./open/train_overlay/made_XDMACLJ.ogg,0,1
7,made_MRORUNV,./open/train_overlay/made_MRORUNV.ogg,1,1
8,made_HWQYKMR,./open/train_overlay/made_HWQYKMR.ogg,0,1
9,made_ZVGCMCB,./open/train_overlay/made_ZVGCMCB.ogg,1,0


In [532]:
res_df = new_df 

for idx, row in new_df.iterrows():
    if row['label_1'] == 0 and row['label_2'] == 0:
        res_df.at[idx, 'label_fake'] = 1
        res_df.at[idx, 'label_real'] = 0
    elif row['label_1'] == 1 and row['label_2'] == 1:
        res_df.at[idx, 'label_fake'] = 0
        res_df.at[idx, 'label_real'] = 1
    else:
        res_df.at[idx, 'label_fake'] = 1
        res_df.at[idx, 'label_real'] = 1

In [533]:
res_df.head(20)

,id,path,label_1,label_2,label_fake,label_real
0,made_ZVPLMJZ,./open/train_overlay/made_ZVPLMJZ.ogg,0,1,1.0,1.0
1,made_WVOSCQM,./open/train_overlay/made_WVOSCQM.ogg,0,1,1.0,1.0
2,made_AWFBVJI,./open/train_overlay/made_AWFBVJI.ogg,1,1,0.0,1.0
3,made_CNOQHMY,./open/train_overlay/made_CNOQHMY.ogg,1,1,0.0,1.0
4,made_JWTCJWJ,./open/train_overlay/made_JWTCJWJ.ogg,1,1,0.0,1.0
5,made_BMGBIUB,./open/train_overlay/made_BMGBIUB.ogg,0,1,1.0,1.0
6,made_XDMACLJ,./open/train_overlay/made_XDMACLJ.ogg,0,1,1.0,1.0
7,made_MRORUNV,./open/train_overlay/made_MRORUNV.ogg,1,1,0.0,1.0
8,made_HWQYKMR,./open/train_overlay/made_HWQYKMR.ogg,0,1,1.0,1.0
9,made_ZVGCMCB,./open/train_overlay/made_ZVGCMCB.ogg,1,0,1.0,1.0


In [536]:
res_df = res_df.drop(columns = ['label_1','label_2'])
res_df.head(20)

,id,path,label_fake,label_real
0,made_ZVPLMJZ,./open/train_overlay/made_ZVPLMJZ.ogg,1.0,1.0
1,made_WVOSCQM,./open/train_overlay/made_WVOSCQM.ogg,1.0,1.0
2,made_AWFBVJI,./open/train_overlay/made_AWFBVJI.ogg,0.0,1.0
3,made_CNOQHMY,./open/train_overlay/made_CNOQHMY.ogg,0.0,1.0
4,made_JWTCJWJ,./open/train_overlay/made_JWTCJWJ.ogg,0.0,1.0
5,made_BMGBIUB,./open/train_overlay/made_BMGBIUB.ogg,1.0,1.0
6,made_XDMACLJ,./open/train_overlay/made_XDMACLJ.ogg,1.0,1.0
7,made_MRORUNV,./open/train_overlay/made_MRORUNV.ogg,0.0,1.0
8,made_HWQYKMR,./open/train_overlay/made_HWQYKMR.ogg,1.0,1.0
9,made_ZVGCMCB,./open/train_overlay/made_ZVGCMCB.ogg,1.0,1.0


In [539]:
import librosa
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import torchmetrics
import os
import warnings
from IPython.display import Audio
warnings.filterwarnings('ignore')
device = torch.device('cpu')

In [693]:
# config
class Config:
    SR = 16000
    N_MFCC = 70
    N_MELS = 70
    # Dataset
    ROOT_FOLDER = './'
    # Training
    N_CLASSES = 2
    BATCH_SIZE = 32
    N_EPOCHS = 15
    LR = 0.0001
    MAX_DURATION = 5 # 최대 시간, 초
    SEED = 27
    PAD_SIZE = MAX_DURATION * SR # 패딩 후 크기 설정 = 샘플 수
    
CONFIG = Config()

In [543]:
def seed_everything(seed):
    import random
    import numpy as np
    import torch

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED)

In [545]:
df = pd.read_csv('./open/train.csv')
df.head()

,id,path,label
0,RUNQPNJF,./train/RUNQPNJF.ogg,real
1,JFAWUOGJ,./train/JFAWUOGJ.ogg,fake
2,RDKEKEVX,./train/RDKEKEVX.ogg,real
3,QYHJDOFK,./train/QYHJDOFK.ogg,real
4,RSPQNHAO,./train/RSPQNHAO.ogg,real


In [547]:
df['path'] = df['path'].apply(lambda x: './open/' + x)
df.head()

,id,path,label
0,RUNQPNJF,./open/./train/RUNQPNJF.ogg,real
1,JFAWUOGJ,./open/./train/JFAWUOGJ.ogg,fake
2,RDKEKEVX,./open/./train/RDKEKEVX.ogg,real
3,QYHJDOFK,./open/./train/QYHJDOFK.ogg,real
4,RSPQNHAO,./open/./train/RSPQNHAO.ogg,real


In [549]:
df = pd.get_dummies(df, columns=["label"], prefix="label", dtype=int)
df.head()

,id,path,label_fake,label_real
0,RUNQPNJF,./open/./train/RUNQPNJF.ogg,0,1
1,JFAWUOGJ,./open/./train/JFAWUOGJ.ogg,1,0
2,RDKEKEVX,./open/./train/RDKEKEVX.ogg,0,1
3,QYHJDOFK,./open/./train/QYHJDOFK.ogg,0,1
4,RSPQNHAO,./open/./train/RSPQNHAO.ogg,0,1


In [551]:
res_df.head()

,id,path,label_fake,label_real
0,made_ZVPLMJZ,./open/train_overlay/made_ZVPLMJZ.ogg,1.0,1.0
1,made_WVOSCQM,./open/train_overlay/made_WVOSCQM.ogg,1.0,1.0
2,made_AWFBVJI,./open/train_overlay/made_AWFBVJI.ogg,0.0,1.0
3,made_CNOQHMY,./open/train_overlay/made_CNOQHMY.ogg,0.0,1.0
4,made_JWTCJWJ,./open/train_overlay/made_JWTCJWJ.ogg,0.0,1.0


In [553]:
df.head()

,id,path,label_fake,label_real
0,RUNQPNJF,./open/./train/RUNQPNJF.ogg,0,1
1,JFAWUOGJ,./open/./train/JFAWUOGJ.ogg,1,0
2,RDKEKEVX,./open/./train/RDKEKEVX.ogg,0,1
3,QYHJDOFK,./open/./train/QYHJDOFK.ogg,0,1
4,RSPQNHAO,./open/./train/RSPQNHAO.ogg,0,1


In [555]:
res_df.shape, df.shape

((20000, 4), (55438, 4))

In [557]:
df_combined = pd.concat([df, res_df], ignore_index=True)
df_combined

,id,path,label_fake,label_real
0,RUNQPNJF,./open/./train/RUNQPNJF.ogg,0.0,1.0
1,JFAWUOGJ,./open/./train/JFAWUOGJ.ogg,1.0,0.0
2,RDKEKEVX,./open/./train/RDKEKEVX.ogg,0.0,1.0
3,QYHJDOFK,./open/./train/QYHJDOFK.ogg,0.0,1.0
4,RSPQNHAO,./open/./train/RSPQNHAO.ogg,0.0,1.0
...,...,...,...,...
75433,made_XZEXGJJ,./open/train_overlay/made_XZEXGJJ.ogg,1.0,0.0
75434,made_VBBDQHW,./open/train_overlay/made_VBBDQHW.ogg,0.0,1.0
75435,made_NRBGPDB,./open/train_overlay/made_NRBGPDB.ogg,1.0,0.0
75436,made_OEERXFB,./open/train_overlay/made_OEERXFB.ogg,1.0,0.0


In [559]:
df_combined.shape

(75438, 4)

In [561]:
df_combined.to_csv('combined_dataframe.csv', index=False)

In [563]:
X = df_combined[["id", "path"]]
y = df_combined[["label_fake", "label_real"]]

In [565]:
X.head(), y.head()

(         id                         path
 0  RUNQPNJF  ./open/./train/RUNQPNJF.ogg
 1  JFAWUOGJ  ./open/./train/JFAWUOGJ.ogg
 2  RDKEKEVX  ./open/./train/RDKEKEVX.ogg
 3  QYHJDOFK  ./open/./train/QYHJDOFK.ogg
 4  RSPQNHAO  ./open/./train/RSPQNHAO.ogg,
    label_fake  label_real
 0         0.0         1.0
 1         1.0         0.0
 2         0.0         1.0
 3         0.0         1.0
 4         0.0         1.0)

In [567]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.35, random_state=CONFIG.SEED)

In [569]:
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [571]:
X_train

,id,path
0,GKFXBMLJ,./open/./train/GKFXBMLJ.ogg
1,made_KMQKEIX,./open/train_overlay/made_KMQKEIX.ogg
2,QTZNNORA,./open/./train/QTZNNORA.ogg
3,JLNUWZZR,./open/./train/JLNUWZZR.ogg
4,UXQNPDLX,./open/./train/UXQNPDLX.ogg
...,...,...
49029,YWDSCJIQ,./open/./train/YWDSCJIQ.ogg
49030,made_BKJYDHG,./open/train_overlay/made_BKJYDHG.ogg
49031,DNTFTYTX,./open/./train/DNTFTYTX.ogg
49032,OOFAWJDL,./open/./train/OOFAWJDL.ogg


In [573]:
root_dir = "."

def load_audio(df, root_dir):
    audios = []
    for i in tqdm(range(df.shape[0])):
        file_path = os.path.join(root_dir, df["path"][i])
        audio, sample_rate = librosa.load(file_path, sr=Config.SR)
        audio_np = np.array(audio)
        audios.append(audio_np)
    return audios

In [575]:
train_audios = load_audio(X_train, root_dir)

100%|████████████████████████████████████| 49034/49034 [02:09<00:00, 379.96it/s]


In [576]:
val_audios = load_audio(X_val, root_dir)

100%|████████████████████████████████████| 26404/26404 [01:14<00:00, 354.95it/s]


In [577]:
import numpy as np

def pad_audio(audio, target_length):
    if len(audio) < target_length:
        pad_width = target_length - len(audio)
        left = pad_width // 2
        right = pad_width - left
        padded_audio = np.pad(audio, (left, right), mode='constant')
    else:
        padded_audio = audio[:target_length]
    return padded_audio

target_length = 16000*5

train_audios_padded = [pad_audio(audio, target_length) for audio in tqdm(train_audios)]
val_audios_padded = [pad_audio(audio, target_length) for audio in tqdm(val_audios)]

100%|███████████████████████████████████| 26404/26404 [00:12<00:00, 2038.80it/s]


In [578]:
def extract_feature(audios, mfcc=True):
    features = []
    if mfcc: #mfcc로 피처를 추출하고 싶을 때
        for audio in tqdm(audios):
            mfcc_feature = librosa.feature.mfcc(y=audio, sr=CONFIG.SR, n_mfcc=CONFIG.N_MFCC)
            mfcc_feature = np.mean(mfcc_feature.T, axis=0)
            features.append(mfcc_feature)
    return np.array(features)

In [579]:
mfcc_X_train_features = extract_feature(train_audios_padded, mfcc=True)
mfcc_X_val_features = extract_feature(val_audios_padded, mfcc=True)

100%|████████████████████████████████████| 26404/26404 [02:25<00:00, 181.27it/s]


In [580]:
np.save('./open/feature/0714mfcc_X_train_features.npy', mfcc_X_train_features)
np.save('./open/feature/0714mfcc_X_val_features.npy', mfcc_X_val_features)

In [695]:
mfcc_X_train_features = np.load('./open/feature/0714mfcc_X_train_features.npy')
mfcc_X_val_features = np.load('./open/feature/0714mfcc_X_val_features.npy')

In [697]:
print("mfcc_X_train_features.shape",mfcc_X_train_features.shape)
#((47122, 13)): (샘플 수, mfcc 계수)
print("mfcc_X_val_features.shape",mfcc_X_val_features.shape)

mfcc_X_train_features.shape (49034, 70)
mfcc_X_val_features.shape (26404, 70)


## 모델 생성

In [700]:
import tensorflow as tf

In [702]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, Activation, LSTM, Reshape
from tensorflow.keras.layers import Bidirectional

def mfcc_cnn_lstm(input_shape, num_classes):
    model = Sequential()

    model.add(Conv1D(32, kernel_size=3, padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))

    model.add(Conv1D(64, kernel_size=3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))

    model.add(Conv1D(128, kernel_size=3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))

    # LSTM 
    # model.add(LSTM(64, return_sequences=True))
    # model.add(LSTM(64))  
    # 결과: 0.39317 -> cnn만을 사용했을 때보다 0.03정도 낮아짐
    
    #LSTM 대신 BiLstm으로 교체해봄 0.3792214873	
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(256)))

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(1, activation='sigmoid'))

    return model

input_shape_mfcc = (70, 1)  # (timesteps, features)
num_classes = CONFIG.N_CLASSES

In [704]:
model_fake = mfcc_cnn_lstm(input_shape_mfcc, num_classes)

In [706]:
model_fake.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [708]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# 데이터 변환 (timesteps, features) 형태로
mfcc_X_train_features_reshaped = mfcc_X_train_features.reshape(mfcc_X_train_features.shape[0], mfcc_X_train_features.shape[1], 1)
mfcc_X_val_features_reshaped = mfcc_X_val_features.reshape(mfcc_X_val_features.shape[0], mfcc_X_val_features.shape[1], 1)

In [710]:
y_train_fake = y_train["label_fake"]
y_train_real = y_train["label_real"]
y_val_fake = y_val["label_fake"]
y_val_real = y_val["label_real"]

In [714]:
# hist_fake = model_fake.fit(mfcc_X_train_features_reshaped, y_train_fake, 
#                  validation_data=(mfcc_X_val_features_reshaped, y_val_fake), 
#                  epochs=15, batch_size=32, callbacks=[es], verbose=1)

In [679]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, Activation, LSTM, Reshape
from tensorflow.keras.layers import Bidirectional

def mfcc_cnn_lstm(input_shape, num_classes):
    model = Sequential()

    model.add(Conv1D(32, kernel_size=3, padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.8))

    model.add(Conv1D(64, kernel_size=3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.8))

    # LSTM 
    # model.add(LSTM(64, return_sequences=True))
    # model.add(LSTM(64))  
    # 결과: 0.39317 -> cnn만을 사용했을 때보다 0.03정도 낮아짐
    
    #LSTM 대신 BiLstm으로 교체해봄 0.3792214873	
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Bidirectional(LSTM(128)))

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(1, activation='sigmoid'))

    return model

input_shape_mfcc = (70, 1)  # (timesteps, features)
num_classes = CONFIG.N_CLASSES

In [681]:
model_real = mfcc_cnn_lstm(input_shape_mfcc, num_classes)
model_real.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [683]:
hist_real = model_real.fit(mfcc_X_train_features_reshaped, y_train_real, 
                 validation_data=(mfcc_X_val_features_reshaped, y_val_real), 
                 epochs=10, batch_size=32, callbacks=[es], verbose=1)

Epoch 1/10
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 80s 51ms/step - accuracy: 0.7649 - loss: 0.4820 - val_accuracy: 0.7426 - val_loss: 0.4799
Epoch 2/10
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 81s 53ms/step - accuracy: 0.8453 - loss: 0.3514 - val_accuracy: 0.8571 - val_loss: 0.3419
Epoch 3/10
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 82s 53ms/step - accuracy: 0.8584 - loss: 0.3303 - val_accuracy: 0.8694 - val_loss: 0.3279
Epoch 4/10
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 81s 53ms/step - accuracy: 0.8608 - loss: 0.3188 - val_accuracy: 0.8740 - val_loss: 0.3134
Epoch 5/10
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 79s 52ms/step - accuracy: 0.8706 - loss: 0.3023 - val_accuracy: 0.8754 - val_loss: 0.2883
Epoch 6/10
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 80s 52ms/step - accuracy: 0.8711 - loss: 0.2963 - val_accuracy: 0.8389 - val_loss: 0.3515
Epoch 7/10
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 80s 52ms/step - accuracy: 0.8779 - loss: 0.2868 - val_accuracy: 0.8491 - val_loss: 0.3291
Epoch 8/10
1533/1533 ━━━━━━━━━━━━━━━━━━━━ 80s 52ms/step - accuracy: 0.8817 -

In [608]:
test = pd.read_csv('./open/test.csv')
print(test.head())

root_dir = "./open"
test = load_audio(test, root_dir)

test_mfcc = extract_feature(test, mfcc=True)

           id                   path
0  TEST_00000  ./test/TEST_00000.ogg
1  TEST_00001  ./test/TEST_00001.ogg
2  TEST_00002  ./test/TEST_00002.ogg
3  TEST_00003  ./test/TEST_00003.ogg
4  TEST_00004  ./test/TEST_00004.ogg


100%|████████████████████████████████████| 50000/50000 [05:15<00:00, 158.30it/s]


In [609]:
import tensorflow as tf

test_dataset = tf.data.Dataset.from_tensor_slices(test_mfcc).batch(CONFIG.BATCH_SIZE)

In [610]:
# def inference(model, test_dataset):
#     predictions = []
#     for features in tqdm(test_dataset):
#         probs = model(features, training=False)
#         predictions.extend(probs.numpy().tolist())
#     return predictions

def inference(model, test_dataset):
    predictions = []
    for features in tqdm(test_dataset):
        probs = model(features, training=False)
        predictions.extend(probs.numpy().tolist())
    return predictions

In [685]:
fake_preds = inference(model_fake, test_dataset)
real_preds = inference(model_real, test_dataset)

100%|███████████████████████████████████████| 1563/1563 [01:47<00:00, 14.54it/s]


In [687]:
submit = pd.read_csv('./open/sample_submission.csv')
submit.head(1)

,id,fake,real
0,TEST_00000,0,0


In [689]:
submit.iloc[:, 1] = fake_preds
submit.iloc[:, 2] = real_preds
submit.head(60)

,id,fake,real
0,TEST_00000,0.492275,0.975452
1,TEST_00001,0.483989,0.960334
2,TEST_00002,0.486202,0.971972
3,TEST_00003,0.468325,0.823141
4,TEST_00004,0.493074,0.988518
5,TEST_00005,0.469889,0.973812
6,TEST_00006,0.461886,0.954981
7,TEST_00007,0.485826,0.911474
8,TEST_00008,0.491043,0.901741
9,TEST_00009,0.474177,0.836065


In [691]:
submit.to_csv('./open/result/0714_5.csv', index=False)